In [1]:
from IPython.display import display
import planner


plan_text = """\
do something c 5h
do something d 6h
"""
controller = planner.PlanController(plan_text, "8-6", True, "4h", "4h")

plan_text2 = """\
do something a 30m
do something b 1h10m
"""
controller2 = planner.PlanController(plan_text2, "13:00")

display(controller.container)
display(controller2.container)

In [2]:
from IPython.display import display
import logger


logs = []
log_controller = logger.LogController(logs)
display(log_controller.container)